In [ ]:
import os
import pandas as pd
import sqlite3

# 1. Dosya Yolu ve Bağlantı Ayarları
current_dir = os.getcwd()
db_name = "database_query_final.db"
# Veritabanı dosyasını çalışma dizinine oluşturuyoruz
db_path = os.path.join(current_dir, db_name)

# Bağlantıyı aç
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# SQLite'da Foreign Key (İlişki) desteğini aktif etmek için gereklidir
cursor.execute("PRAGMA foreign_keys = ON;")

# ---------------------------------------------------------
# 2. TABLOLARI OLUŞTURMA (CREATE TABLE)
# ---------------------------------------------------------

# Önce tablolar varsa silelim (Hata almamak için - Temiz başlangıç)
cursor.execute("DROP TABLE IF EXISTS personeller")
cursor.execute("DROP TABLE IF EXISTS bolumler")
cursor.execute("DROP TABLE IF EXISTS gorevler")

# Görevler Tablosu SQL'i
sql_gorevler = """
CREATE TABLE gorevler (
    gorevNo INT PRIMARY KEY,
    gorevAdi VARCHAR(50) NOT NULL
);
"""

# Bölümler Tablosu SQL'i
sql_bolumler = """
CREATE TABLE bolumler (
    bolumNo INT PRIMARY KEY,
    bolumAdi VARCHAR(50) NOT NULL
);
"""
# Personeller Tablosu SQL'i (İlişkiler burada tanımlanıyor)
sql_personeller = """
CREATE TABLE personeller (
    personelNo INT PRIMARY KEY,
    personelAdi VARCHAR(50),
    personelSoyadi VARCHAR(50),
    personelGorev INT,
    personelBolum INT,
    personelTel VARCHAR(15),
    personelMaas REAL,
    FOREIGN KEY (personelGorev) REFERENCES gorevler(gorevNo),
    FOREIGN KEY (personelBolum) REFERENCES bolumler(bolumNo)
);
"""
# Komutları çalıştır (Tabloları yarat)
cursor.execute(sql_gorevler)
cursor.execute(sql_bolumler)
cursor.execute(sql_personeller)
print("Tablolar başarıyla oluşturuldu.")

# ---------------------------------------------------------
# 3. VERİLERİ EKLEME (INSERT INTO)
# ---------------------------------------------------------

# Toplu veri girişi için listeler hazırlıyoruz
data_gorevler = [
    (1, 'Memur'), (2, 'Mühendis'), (3, 'Doktor'), 
    (4, 'Muhasebeci'), (5, 'Pazarlamacı'), (6, 'İşçi'), (7, 'Hizmetli')
]
data_bolumler = [
    (1, 'Muhasebe'), (2, 'Satış'), (3, 'Arge'), 
    (4, 'Üretim'), (5, 'İnsan Kaynakları')
]
data_personeller = [
    (1, 'Ali', 'Kaya', 2, 3, '555', 10000),
    (2, 'Taner', 'Kayalı', 2, 3, '554', 12000),
    (3, 'Hasan', 'Güneş', 2, 4, '551', 7000),
    (4, 'Göknur', 'Tozar', 1, 5, '550', 9000),
    (5, 'Zeynep', 'Yaş', 4, 1, '544', 7500),
    (6, 'Elif', 'Göze', 5, 2, '543', 6500),
    (7, 'İsa', 'Nur', 6, 4, '542', 6400),
    (8, 'Ali', 'Say', 6, 4, '541', 5000),
    (9, 'Yusuf', 'Taş', 7, 4, '541', 5000),
    (10, 'Halit', 'Söz', 7, None, '535', 7000) # Python'da NULL yerine None kullanılır
]
# executemany: Birden çok satırı tek komutla ekler (Daha performanslıdır)
cursor.executemany("INSERT INTO gorevler VALUES (?,?)", data_gorevler)
cursor.executemany("INSERT INTO bolumler VALUES (?,?)", data_bolumler)
cursor.executemany("INSERT INTO personeller VALUES (?,?,?,?,?,?,?)", data_personeller)
print("Veriler başarıyla eklendi.")
# ---------------------------------------------------------
# 4. KAYDETME VE KONTROL
# ---------------------------------------------------------

# Değişiklikleri veritabanına işle (Commit)
connection.commit()

# Bağlantıyı kapat
#SORGU FONKSIYON TANIMLA
def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None

Tablolar başarıyla oluşturuldu.
Veriler başarıyla eklendi.


In [116]:
print("PERSONELLER TABLOSU: \n")
run_query("select*from personeller")

PERSONELLER TABLOSU: 

   personelNo personelAdi personelSoyadi  personelGorev  personelBolum  \
0           1         Ali           Kaya              2            3.0   
1           2       Taner         Kayalı              2            3.0   
2           3       Hasan          Güneş              2            4.0   
3           4      Göknur          Tozar              1            5.0   
4           5      Zeynep            Yaş              4            1.0   
5           6        Elif           Göze              5            2.0   
6           7         İsa            Nur              6            4.0   
7           8         Ali            Say              6            4.0   
8           9       Yusuf            Taş              7            4.0   
9          10       Halit            Söz              7            NaN   

  personelTel  personelMaas  
0         555       10000.0  
1         554       12000.0  
2         551        7000.0  
3         550        9000.0  
4         54

In [117]:
print("BOLUMLER TABLOSU: \n")
run_query("select*from bolumler")

BOLUMLER TABLOSU: 

   bolumNo          bolumAdi
0        1          Muhasebe
1        2             Satış
2        3              Arge
3        4            Üretim
4        5  İnsan Kaynakları


In [118]:
print("BOLUMLER GOREVLER: \n")
run_query("select*from gorevler")

BOLUMLER GOREVLER: 

   gorevNo     gorevAdi
0        1        Memur
1        2     Mühendis
2        3       Doktor
3        4   Muhasebeci
4        5  Pazarlamacı
5        6         İşçi
6        7     Hizmetli


FİNALDE ÇIKMIŞ SORULAR

In [119]:
#Zeynep Yaş isimli personelin bölümünü İnsan kaynakları numarasını 5543885858, maaşını 10000 olarak güncelleyen sql sorgusunu yazınız
try:
    cursor.execute("""
               update personeller
               set
                personelTel=868686,
                personelBolum=5,
                personelMaas=10000
               where
                personelAdi = 'Zeynep' and personelSoyadi='Yaş';
    """)
except Exception as e:
    print(e)

In [120]:
#Personel tablosuna doğum tarihi adında bir sütun ekleyiniz, girilecek tarihin 1960 ile 2004 arasında olmasını sağlayınız.
try:
    cursor.execute("""
    alter table personeller
               add column dogumTarihi text
               check (dogumTarihi BETWEEN '1960-01-01' AND '2004-12-31');
    """)
except Exception as e:
    print(e)

In [121]:
#Personel tablosuna üretim departmanına iki adet işçi ekleyen sql sorgusunu yazınız.

try:
    cursor.execute("""
            insert into
               personeller(personelAdi, personelSoyadi, personelGorev, personelBolum, personelTel, personelMaas)
               values 
                ('Ahmet', 'Demir', 6, 4, '556', 5000),
                ('Mehmet', 'Çelik', 6, 4, '557', 5000);
   """)
except Exception as e:
    print(e)

In [122]:
#Üretim departmanında ismi İsa olan memurları silen sql sorgusunu yazınız.
try:
    cursor.execute("""
            delete from personeller
                   where personelAdi = 'İsa'
                    and personelBolum = (select bolumNo from bolumler where bolumAdi='Üretim')
                    and personelGorev = (select gorevNo from gorevler where gorevAdi ='Memur');
    """)
except Exception as e:
    print(e)

In [123]:
#Tüm personellerin adı, soyadı, görevini, departmanını ve maaşını görüntüleyen sql sorgusunu yazınız.
run_query("""
    select
        p.personelAdi, 
        p.personelSoyadi, 
        b.bolumAdi,      
        g.gorevAdi,     
        p.personelMaas
    from personeller as p
    inner join bolumler as b on p.personelBolum = b.bolumNo
    inner join gorevler as g on p.personelGorev = g.gorevNo;
""")

   personelAdi personelSoyadi          bolumAdi     gorevAdi  personelMaas
0          Ali           Kaya              Arge     Mühendis       10000.0
1        Taner         Kayalı              Arge     Mühendis       12000.0
2        Hasan          Güneş            Üretim     Mühendis        7000.0
3       Göknur          Tozar  İnsan Kaynakları        Memur        9000.0
4       Zeynep            Yaş  İnsan Kaynakları   Muhasebeci       10000.0
5         Elif           Göze             Satış  Pazarlamacı        6500.0
6          İsa            Nur            Üretim         İşçi        6400.0
7          Ali            Say            Üretim         İşçi        5000.0
8        Yusuf            Taş            Üretim     Hizmetli        5000.0
9        Ahmet          Demir            Üretim         İşçi        5000.0
10      Mehmet          Çelik            Üretim         İşçi        5000.0


In [124]:
#Tüm maaşların ortalamasından daha düşük maaş alan personellerin adı, soyadı, bölümü, görevini ve maaşını gösteren sql sorgusunu yazınız.
run_query("""
    select
        p.personelAdi, 
        p.personelSoyadi, 
        b.bolumAdi,      
        g.gorevAdi,     
        p.personelMaas
    from personeller as p
    inner join
          bolumler as b on p.personelBolum = b.bolumNo
    inner join
          gorevler as g on p.personelGorev = g.gorevNo
    WHERE
          p.personelMaas < (SELECT AVG(personelMaas) FROM personeller);""")
#bu sorgu JOIN ALIAS ve SUBQUERY içeriyor


  personelAdi personelSoyadi bolumAdi     gorevAdi  personelMaas
0       Hasan          Güneş   Üretim     Mühendis        7000.0
1        Elif           Göze    Satış  Pazarlamacı        6500.0
2         İsa            Nur   Üretim         İşçi        6400.0
3         Ali            Say   Üretim         İşçi        5000.0
4       Yusuf            Taş   Üretim     Hizmetli        5000.0
5       Ahmet          Demir   Üretim         İşçi        5000.0
6      Mehmet          Çelik   Üretim         İşçi        5000.0


In [125]:
#Yusuf Taş isimli personel ile görevi, bölümü ve maaşı aynı personelleri listeleyen sql sorgusunu yazınız.
run_query("""
    select * from personeller
    where
          personelMaas = (select personelMaas from personeller where personelAdi = 'Yusuf' and personelSoyadi='Taş')
    and
          personelGorev = (select personelGorev from personeller where personelAdi = 'Yusuf' and personelSoyadi='Taş')
    and
          personelBolum = (select personelBolum from personeller where personelAdi = 'Yusuf' and personelSoyadi='Taş');
""")

   personelNo personelAdi personelSoyadi  personelGorev  personelBolum  \
0           9       Yusuf            Taş              7              4   

  personelTel  personelMaas dogumTarihi  
0         541        5000.0        None  


In [126]:
#Yusuf Taş isimli personel ile görevi, bölümü ve maaşı aynı personelleri listeleyen sql sorgusunu yazınız.
run_query("""
    SELECT 
        p.personelAdi, 
        g.gorevAdi, 
        b.bolumAdi, 
        p.personelMaas 
    FROM personeller AS p
    INNER JOIN gorevler AS g ON p.personelGorev = g.gorevNo  
    INNER JOIN bolumler AS b ON p.personelBolum = b.bolumNo  
    WHERE 
        p.personelMaas = (SELECT personelMaas FROM personeller WHERE personelAdi = 'Yusuf' AND personelSoyadi = 'Taş')
        AND 
        p.personelBolum = (SELECT personelBolum FROM personeller WHERE personelAdi = 'Yusuf' AND personelSoyadi = 'Taş')
        AND 
        p.personelGorev = (SELECT personelGorev FROM personeller WHERE personelAdi = 'Yusuf' AND personelSoyadi = 'Taş');
        
""")

  personelAdi  gorevAdi bolumAdi  personelMaas
0       Yusuf  Hizmetli   Üretim        5000.0


In [127]:
#Maaşı genel maaş ortalamasının altında olanların maaşına %5 oranında zam yaparak güncelleyen sql kodunu yazınız
try:
    cursor.execute("""
            update personeller set personelMaas = personelMaas+(personelMaas*0.05)
            where personelMaas<(select avg(personelMaas) from personeller);
    """)
except Exception as e:
    print(e)

In [128]:
#9 numaralı personel ile aynı görevde çalışan, fakat maaşı 7 numaralı personelin maaşından daha düşük olan çalışanların; adını, soyadını, görevini ve maaşını listele
run_query("""
    SELECT personelAdi, personelSoyadi, personelGorev, personelMaas
    FROM personeller
    WHERE personelGorev = (SELECT personelGorev FROM personeller WHERE personelNo = 9)
    AND personelMaas < (SELECT personelMaas FROM personeller WHERE personelNo = 7);
""")

  personelAdi personelSoyadi  personelGorev  personelMaas
0       Yusuf            Taş              7        5250.0


In [129]:
# Tüm bölümlerin ortalama ücreti göz önünde bulundurularak en düşük ortalama ücretten daha az ücret alan personelin adını, bölümünü ve maaşını listeleyen sql sorgusunu yazınız.
run_query("""
    select p.personelAdi, b.bolumAdi, p.personelMaas
    from personeller as p
    inner join bolumler as b on p.personelBolum = b.bolumNo
    where personelMaas<(
          select avg(personelMaas)
          from personeller
          where personelBolum is not null
          group by personelBolum
          order by avg(personelMaas) asc
          limit 1
          );
""")

  personelAdi bolumAdi  personelMaas
0         Ali   Üretim        5250.0
1       Yusuf   Üretim        5250.0
2       Ahmet   Üretim        5250.0
3      Mehmet   Üretim        5250.0


In [ ]:
#personel tablosuna üretim departmanına iki adet işçi ekleyen sorgu

In [ ]:
#bunlardan birini çıkaran sorgu

In [ ]:
#tüm personellerin adı soyadı görevi ve departmanı

In [ ]:
#en yüksek maaş alan personel adı soyadı bölümü ve görevini gösteren

In [ ]:
#ali isimli personel ile görevi ve ücreti aynı personelleri listele

In [ ]:
#tüm personelin maaşına %4  oranında zam yaparak güncelle

In [ ]:
#bölümlere göre çalışan sayılarını gösteren sorgu

In [ ]:
#tüm bölümlerin ortalama ücreti göz önünde bulundurularak en düşük ortalama ücretten düşük ücret alan personelin adını bölümünü maaşını listeleyen sqlsorgu

In [130]:
run_query('select*from bolumler')
run_query('select*from gorevler')


   bolumNo          bolumAdi
0        1          Muhasebe
1        2             Satış
2        3              Arge
3        4            Üretim
4        5  İnsan Kaynakları
   gorevNo     gorevAdi
0        1        Memur
1        2     Mühendis
2        3       Doktor
3        4   Muhasebeci
4        5  Pazarlamacı
5        6         İşçi
6        7     Hizmetli


In [131]:
connection.commit()
